# 🧠 Complete ASD Screening ML Model Training v2

## Master Dataset Training with Sample Weighting

This notebook implements the **complete ML training pipeline v2** with:
- ✅ Master dataset (real + synthetic data)
- ✅ Sample weighting (real data > synthetic)
- ✅ Age-normalized feature engineering
- ✅ Proper train/validation/test split
- ✅ Test evaluation on real data only
- ✅ Model calibration and comprehensive evaluation

### Key Improvements from v1:
- **Sample Weighting**: Real data weighted 3.3× higher than synthetic
- **Proper Validation**: Separate validation set for model selection
- **Real Data Testing**: Final evaluation only on real clinical data
- **Reproducible**: Fixed random seeds for reproducibility

---


## Step 1: Setup and Install Libraries


In [ ]:
# Install required packages (Google Colab)
# Skip this if using local Jupyter
!pip install pandas numpy scikit-learn xgboost matplotlib seaborn scipy joblib -q

print("✅ All packages installed!")


In [ ]:
# Import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
from sklearn.calibration import CalibratedClassifierCV
from scipy import stats
from scipy.stats import mannwhitneyu, pearsonr

# Google Colab file upload
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully!")
print(f"Running in {'Google Colab' if IN_COLAB else 'Local Jupyter'}")


## Step 2: Load Master Dataset


In [ ]:
# Load master training dataset
if IN_COLAB:
    # Upload file in Colab
    uploaded = files.upload()
    df = pd.read_csv('master_training_dataset.csv')
else:
    # Load from local file
    df = pd.read_csv('../senseai_backend/master_training_dataset.csv')

print(f"📊 Dataset loaded: {len(df)} rows, {len(df.columns)} columns")
print(f"\nData sources: {df['data_source'].value_counts().to_dict()}")
print(f"\nGroups: {df['group'].value_counts().to_dict()}")
print(f"\nFirst few rows:")
df.head()


## Step 3: Data Exploration and Preprocessing


In [ ]:
# Explore data
print("📊 Data Info:")
print(df.info())

print("\n" + "="*60)
print("Missing Values:")
missing = df.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])

print("\n" + "="*60)
print("Target Distribution:")
print(df['group'].value_counts())

print("\n" + "="*60)
print("Data Source Distribution:")
print(df['data_source'].value_counts())

print("\n" + "="*60)
print("Age Distribution:")
if 'age_months' in df.columns:
    print(df['age_months'].describe())


In [ ]:
# Remove rows with missing group (target variable)
df = df[df['group'].notna()].copy()

# Separate real and synthetic data
real_data = df[df['data_source'] == 'real'].copy()
synthetic_data = df[df['data_source'].isin(['synthetic_asd', 'synthetic_td'])].copy()

print(f"📊 Real data: {len(real_data)} rows")
print(f"📊 Synthetic data: {len(synthetic_data)} rows")
print(f"\nReal data groups: {real_data['group'].value_counts().to_dict()}")
print(f"Synthetic data groups: {synthetic_data['group'].value_counts().to_dict()}")

# Check for missing values in key columns
print(f"\nMissing values in real data:")
print(real_data.isnull().sum()[real_data.isnull().sum() > 0].head(10))


## Step 4: Feature Engineering

### 4.1: Calculate Derived Features


In [ ]:
# Calculate derived features if they don't exist
print("🔧 Calculating derived features...")

# Switch cost (if not present)
if 'switch_cost_ms' not in df.columns or df['switch_cost_ms'].isna().all():
    if 'avg_rt_post_switch_correct_ms' in df.columns and 'avg_rt_pre_switch_ms' in df.columns:
        df['switch_cost_ms'] = df['avg_rt_post_switch_correct_ms'] - df['avg_rt_pre_switch_ms']
        print("   ✅ Calculated: switch_cost_ms")

# Accuracy drop (if not present)
if 'accuracy_drop_percent' not in df.columns or df['accuracy_drop_percent'].isna().all():
    if 'pre_switch_accuracy' in df.columns and 'post_switch_accuracy' in df.columns:
        df['accuracy_drop_percent'] = df['pre_switch_accuracy'] - df['post_switch_accuracy']
        print("   ✅ Calculated: accuracy_drop_percent")

# Commission error rate (if not present)
if 'commission_error_rate' not in df.columns or df['commission_error_rate'].isna().all():
    if 'nogo_accuracy' in df.columns:
        df['commission_error_rate'] = 100 - df['nogo_accuracy']
        print("   ✅ Calculated: commission_error_rate")

# RT variability (if not present)
if 'rt_variability' not in df.columns or df['rt_variability'].isna().all():
    if 'avg_reaction_time_ms' in df.columns:
        # Use coefficient of variation as proxy
        df['rt_variability'] = df['avg_reaction_time_ms'] * 0.15  # Approximate
        print("   ✅ Estimated: rt_variability")

print("\n✅ Derived features calculated!")


### 4.2: Age Normalization (Z-Scores)


In [ ]:
# Age normalization using control group norms
print("🔧 Performing age normalization...")

# Get control group data (for establishing norms)
control_df = df[df['group'] == 'typically_developing'].copy()

# Features to normalize (age-sensitive features)
features_to_normalize = [
    'switch_cost_ms',
    'perseverative_error_rate_post_switch',
    'commission_error_rate',
    'rt_variability',
    'post_switch_accuracy',
    'nogo_accuracy',
    'avg_rt_pre_switch_ms',
    'avg_rt_post_switch_correct_ms',
    'avg_rt_go_ms',
    'accuracy_drop_percent',
    'go_accuracy',
    'overall_accuracy'
]

# Filter to features that exist in dataset
features_to_normalize = [f for f in features_to_normalize if f in df.columns]

if len(features_to_normalize) > 0 and len(control_df) > 0:
    print(f"Normalizing {len(features_to_normalize)} features using control group norms...")
    
    for feature in features_to_normalize:
        z_scores = []
        
        for idx, row in df.iterrows():
            age = row.get('age_months', 36)
            value = row[feature]
            
            # Skip if missing
            if pd.isna(value) or pd.isna(age):
                z_scores.append(0)
                continue
            
            # Find age-matched controls (±6 months window)
            age_band_controls = control_df[
                (control_df['age_months'] >= age - 6) & 
                (control_df['age_months'] <= age + 6)
            ]
            
            if len(age_band_controls) > 1:
                # Use age-matched controls
                mean_val = age_band_controls[feature].mean()
                std_val = age_band_controls[feature].std()
                z_score = (value - mean_val) / std_val if std_val > 0 else 0
            elif len(control_df) > 0:
                # Fallback to all controls
                mean_val = control_df[feature].mean()
                std_val = control_df[feature].std()
                z_score = (value - mean_val) / std_val if std_val > 0 else 0
            else:
                z_score = 0
            
            z_scores.append(z_score)
        
        # Create z-score column
        df[f'{feature}_zscore'] = z_scores
        print(f"   ✅ Normalized: {feature} → {feature}_zscore")
    
    print(f"\n✅ Age normalization completed!")
else:
    print("⚠️  Skipping age normalization (no control data or features)")


In [ ]:
# Define feature candidates (aligned with your existing model)
FEATURE_CANDIDATES = [
    # Age
    "age_months",
    
    # Cognitive Flexibility (DCCS) - Raw
    "post_switch_accuracy",
    "pre_switch_accuracy",
    "switch_cost_ms",
    "avg_rt_pre_switch_ms",
    "avg_rt_post_switch_correct_ms",
    "accuracy_drop_percent",
    "perseverative_error_rate_post_switch",
    "cognitive_flexibility_score",
    
    # Cognitive Flexibility (DCCS) - Z-scores
    "post_switch_accuracy_zscore",
    "switch_cost_ms_zscore",
    "avg_rt_pre_switch_ms_zscore",
    "avg_rt_post_switch_correct_ms_zscore",
    "accuracy_drop_percent_zscore",
    "perseverative_error_rate_post_switch_zscore",
    
    # Inhibitory Control (Go/No-Go) - Raw
    "go_accuracy",
    "nogo_accuracy",
    "commission_error_rate",
    "omission_error_rate",
    "inhibition_failure_rate",
    
    # Inhibitory Control (Go/No-Go) - Z-scores
    "nogo_accuracy_zscore",
    "go_accuracy_zscore",
    "commission_error_rate_zscore",
    
    # Reaction Time Metrics
    "avg_reaction_time_ms",
    "rt_variability",
    "rt_range",
    "rt_variability_zscore",
    "avg_rt_go_ms",
    "avg_rt_go_ms_zscore",
    
    # Behavioral Observations
    "attention_level",
    "engagement_level",
    "frustration_tolerance",
    "instruction_following",
    
    # Social Communication
    "social_communication_score",
    "social_responsiveness_score",
    "joint_attention_score",
    
    # Task Performance
    "completion_time_sec",
    "overall_accuracy",
    "overall_accuracy_zscore",
]

# Filter to features that exist in dataset
available_features = [f for f in FEATURE_CANDIDATES if f in df.columns]
missing_features = [f for f in FEATURE_CANDIDATES if f not in df.columns]

print(f"✅ Available features: {len(available_features)}/{len(FEATURE_CANDIDATES)}")
print(f"\nAvailable features:")
for feat in available_features:
    print(f"  - {feat}")

if missing_features:
    print(f"\n⚠️ Missing features (will be skipped):")
    for feat in missing_features[:10]:  # Show first 10
        print(f"  - {feat}")
    if len(missing_features) > 10:
        print(f"  ... and {len(missing_features) - 10} more")

# Use available features
FEATURE_COLUMNS = available_features
TARGET = "group"

print(f"\n✅ Using {len(FEATURE_COLUMNS)} features for training")


In [ ]:
# Combine real and synthetic data
all_data = pd.concat([real_data, synthetic_data], ignore_index=True)

# Select features and target
X = all_data[FEATURE_COLUMNS].copy()
y = all_data[TARGET].copy()
data_source = all_data['data_source'].copy()

# Handle missing values: Fill with median for numeric features
print("🔧 Handling missing values...")
for col in FEATURE_COLUMNS:
    if X[col].dtype in ['float64', 'int64']:
        missing_count = X[col].isnull().sum()
        if missing_count > 0:
            median_val = X[col].median()
            if pd.isna(median_val):
                median_val = 0  # Fallback
            X[col] = X[col].fillna(median_val)
            print(f"   Filled {col}: {missing_count} missing → median={median_val:.2f}")

# Check remaining missing values
remaining_missing = X.isnull().sum().sum()
print(f"\n✅ Remaining missing values: {remaining_missing}")

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"\n✅ Target encoding:")
for i, class_name in enumerate(le.classes_):
    print(f"  {class_name} → {i}")

print(f"\n✅ Feature matrix shape: {X.shape}")
print(f"✅ Target distribution:")
print(pd.Series(y).value_counts())


## Step 7: Create Sample Weights (Real > Synthetic)


In [ ]:
# Create sample weights: Real data = 1.0, Synthetic = 0.3
sample_weights = np.where(data_source == 'real', 1.0, 0.3)

print(f"📊 Sample weights distribution:")
print(f"  Real data (weight=1.0): {np.sum(sample_weights == 1.0)} samples")
print(f"  Synthetic data (weight=0.3): {np.sum(sample_weights == 0.3)} samples")
print(f"\n  Total weighted samples: {np.sum(sample_weights):.1f}")
print(f"  Effective real samples: {np.sum(sample_weights == 1.0)}")
print(f"  Effective synthetic samples: {np.sum(sample_weights == 0.3) * 0.3:.1f}")


## Step 8: Train/Validation/Test Split


In [ ]:
# Split REAL data into train/val/test (70/15/15)
real_indices = data_source[data_source == 'real'].index
real_X = X.loc[real_indices]
real_y = y_encoded[real_indices]

# First split: 70% train, 30% temp (for val+test)
X_train_real, X_temp, y_train_real, y_temp = train_test_split(
    real_X, real_y, 
    test_size=0.3, 
    random_state=42, 
    stratify=real_y
)

# Second split: 50% val, 50% test (from temp)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

# Add ALL synthetic data to training set
synthetic_indices = data_source[data_source != 'real'].index
X_train_synthetic = X.loc[synthetic_indices]
y_train_synthetic = y_encoded[synthetic_indices]
weights_synthetic = sample_weights[synthetic_indices]

# Combine real training data with synthetic
X_train = pd.concat([X_train_real, X_train_synthetic], ignore_index=True)
y_train = np.concatenate([y_train_real, y_train_synthetic])
weights_train = np.concatenate([
    np.ones(len(y_train_real)),  # Real data = 1.0
    weights_synthetic  # Synthetic = 0.3
])

print(f"📊 Final Split:")
print(f"  Training: {len(X_train)} samples ({len(X_train_real)} real + {len(X_train_synthetic)} synthetic)")
print(f"  Validation: {len(X_val)} samples (real only)")
print(f"  Test: {len(X_test)} samples (real only)")
print(f"\n  Training target distribution:")
print(f"    {pd.Series(le.inverse_transform(y_train)).value_counts().to_dict()}")
print(f"\n  Validation target distribution:")
print(f"    {pd.Series(le.inverse_transform(y_val)).value_counts().to_dict()}")
print(f"\n  Test target distribution:")
print(f"    {pd.Series(le.inverse_transform(y_test)).value_counts().to_dict()}")


## Step 9: Feature Scaling


In [ ]:
# Scale features
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Features scaled:")
print(f"  Training: {X_train_scaled.shape}")
print(f"  Validation: {X_val_scaled.shape}")
print(f"  Test: {X_test_scaled.shape}")
print(f"\n  Feature means (should be ~0): {X_train_scaled.mean(axis=0)[:5]}")
print(f"  Feature stds (should be ~1): {X_train_scaled.std(axis=0)[:5]}")


## Step 10: Train Logistic Regression Model


In [ ]:
# Train Logistic Regression with sample weights
model = LogisticRegression(
    max_iter=2000,
    class_weight='balanced',  # Handle class imbalance
    random_state=42,
    solver='lbfgs'  # Good for small-medium datasets
)

print("🔄 Training model...")
model.fit(X_train_scaled, y_train, sample_weight=weights_train)

print("✅ Model trained successfully!")
print(f"\nModel coefficients shape: {model.coef_.shape}")
print(f"Model intercept: {model.intercept_}")
print(f"Number of features: {len(FEATURE_COLUMNS)}")


## Step 11: Validation Set Evaluation


In [ ]:
# Predict on validation set
y_val_pred = model.predict(X_val_scaled)
y_val_proba = model.predict_proba(X_val_scaled)[:, 1]

# Classification report
print("📊 VALIDATION SET RESULTS:")
print("=" * 60)
print(classification_report(y_val, y_val_pred, target_names=le.classes_))

# Confusion matrix
cm = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Validation Set Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# ROC-AUC
if len(le.classes_) == 2:
    auc = roc_auc_score(y_val, y_val_proba)
    print(f"\nROC-AUC Score: {auc:.4f}")
    
    fpr, tpr, _ = roc_curve(y_val, y_val_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc:.4f})', linewidth=2)
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Validation Set ROC Curve')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


## Step 12: FINAL TEST SET EVALUATION (Real Data Only)

**This is what you report in your thesis!**


In [ ]:
# Predict on test set (REAL DATA ONLY - this is what you report!)
y_test_pred = model.predict(X_test_scaled)
y_test_proba = model.predict_proba(X_test_scaled)[:, 1]

print("🎯 FINAL TEST SET RESULTS (REAL DATA ONLY)")
print("=" * 60)
print("This is what you report in your thesis!")
print("=" * 60)
print(classification_report(y_test, y_test_pred, target_names=le.classes_))

# Confusion matrix
cm_test = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Greens', 
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Test Set Confusion Matrix (Real Data Only)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# ROC-AUC
if len(le.classes_) == 2:
    auc_test = roc_auc_score(y_test, y_test_proba)
    print(f"\n🎯 FINAL TEST ROC-AUC Score: {auc_test:.4f}")
    
    fpr, tpr, _ = roc_curve(y_test, y_test_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc_test:.4f})', linewidth=2)
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Test Set ROC Curve (Real Data Only)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Feature importance (coefficients)
feature_importance = pd.DataFrame({
    'feature': FEATURE_COLUMNS,
    'coefficient': model.coef_[0],
    'abs_coefficient': np.abs(model.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

print("\n📊 Top 15 Most Important Features:")
print(feature_importance.head(15).to_string(index=False))
